Format 1 is just a single column

It just includes the fields:
- property tax
- interest
- total revenues
- capital projects
- revenues over expenditures
- fund balance beginning of year
- fund balance end of year

<img src='format_examples/format_1.png' height='500px'>

In [198]:
import pandas as pd
import re

In [199]:
DOC_YEAR = 1997
DOC_NUM = 29
DOC_PAGE = 20

DATABASE_FIELDS = ['year', 'tif_number', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text']

In [200]:
csv_path = f'../../parsed_pdfs/{DOC_YEAR}_{DOC_NUM}.csv'
tif_text = pd.read_csv(csv_path, header=None, names=DATABASE_FIELDS)

In [201]:
tif_text

,year,tif_number,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1997,29,0,1,1,1,1,857,519,193,70,96.76,1997
1,1997,29,0,1,1,1,2,1077,520,317,69,96.65,Annual
2,1997,29,0,1,1,1,3,1421,520,301,90,95.74,Report
3,1997,29,0,2,1,1,1,894,777,786,92,90.20,Michigan/Cermak
4,1997,29,0,2,1,2,1,668,879,662,90,96.07,Redevelopment
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15565,1997,29,76,8,1,3,10,1521,1896,85,34,95.62,the
15566,1997,29,76,8,1,3,11,1644,1896,178,34,96.21,blocks
15567,1997,29,76,8,1,3,12,1859,1896,180,34,96.31,within
15568,1997,29,76,8,1,3,13,2077,1896,85,34,96.06,the


In [202]:
# Find the text for specifically our page
page = tif_text[tif_text['page_num'] == DOC_PAGE]

In [203]:
# Add another column to every row that is the center of the recognized text
def find_center_x(row):
	return row['left'] + row['width'] / 2

def find_center_y(row):
	return row['top'] + row['height'] / 2

col_x = page.apply(find_center_x, axis=1)
col_y = page.apply(find_center_y, axis=1)

page = page.assign(center_x=col_x.values)
page = page.assign(center_y=col_y.values)

In [204]:
# Drop columns that don't help us for this task
USELESS_COLS = ['year', 'tif_number', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num']
page = page.drop(USELESS_COLS, axis=1)

In [205]:
print(page.to_string())

      left   top  width  height   conf             text  center_x  center_y
3630   923   199    117      30  46.68             City     981.5     214.0
3631  1072   199     57      30  91.80               OF    1100.5     214.0
3632  1164   199    230      44  95.25         CHICAGO,    1279.0     221.0
3633  1442   199    235      31  92.96         ILLINOIS    1559.5     214.5
3634   740   248    451      32  90.97  MICHIGAN-CERMAK     965.5     264.0
3635  1224   248    395      32  92.18    REDEVELOPMENT    1421.5     264.0
3636  1654   248    207      32  88.61          PROJECT    1757.5     264.0
3637   772   349    266      31  95.62        STATEMENT     905.0     364.5
3638  1071   350     57      30  94.62               OF    1099.5     365.0
3639  1161   349    263      38  93.68        REVENUES,    1292.5     368.0
3640  1469   349    359      31  96.14     EXPENDITURES    1648.5     364.5
3641   889   399     89      31  92.65              AND     933.5     414.5
3642  1013  

In [206]:
# Create a dictionary of 'lines' in the document. A line is defined as a group
# of words that are all within 15px along the y-axis of each other.

# To do this, first create an empty dictionary and simply sort the entire dataframe by center_y

# Initially, put all of the items that share the same center_y into the same
# list (inside the dict) keyed by their average

# Iteratively join groups that have an average closer than 15px until there are no merges

In [207]:
page.sort_values(['center_y'])

line_dict = {}

for _, row in page.iterrows():

	if row['center_y'] not in line_dict:
		line_dict[row['center_y']] = []

	line_dict[row['center_y']].append(row.to_dict())


In [208]:
# Now iteratively reduce the list

REDUCED = True
merges = 0
while REDUCED:

	REDUCED = False

	current_keys = sorted(list(line_dict.keys()))

	for i in range(len(current_keys) - 1):
		if not REDUCED and abs(float(current_keys[i]) - float(current_keys[i+1])) <= 15:

			REDUCED = True
			merges += 1

			avg = (current_keys[i] + current_keys[i+1]) / 2

			
			line_dict[avg] = line_dict[current_keys[i]] + line_dict[current_keys[i+1]]

			# Get rid of the original keys
			del line_dict[current_keys[i]]
			del line_dict[current_keys[i+1]]


merges

32

In [209]:
document_line_dict = {}

for level in sorted(list(line_dict.keys())):
	line_dict[level].sort(key=lambda r: r['left'])
	document_line_dict[level] = ''
	for word in line_dict[level]:
		document_line_dict[level] += word['text'] + ' '
	
	if len(line_dict) > 0:
		document_line_dict[level] = document_line_dict[level][:-1]

In [210]:
# For this format, specifically, it looks like there are only specific features we want
document_form_labels = ['property tax', 'interest', 'total revenues', 'capital projects', 'revenues over expenditures', 'fund balance beginning of year', 'fund balance end of year']

In [211]:
table_form = {
	'property tax': -1,
	'interest': -1,
	'total revenues': -1,
	'capital projects': -1,
	'revenues over expenditures': -1,
	'fund balance beginning of year': -1,
	'fund balance end of year': -1,
}

In [212]:
# Go through each line, lowercase it, and remove non-alphanumerics
for line, string in document_line_dict.items():
	document_line_dict[line] = re.sub(r'[^a-z0-9 _]+', '', string.lower())

In [213]:
document_line_dict

{217.625: 'city of chicago illinois',
 264.0: 'michigancermak redevelopment project',
 366.375: 'statement of revenues expenditures',
 414.75: 'and changes in fund balance',
 464.625: 'year ended december 31 1997',
 663.5: 'revenues',
 716.75: 'property tax 173246',
 765.5: 'interest 26247',
 867.125: 'total revenues 199493',
 967.5: 'expenditures',
 1019.125: 'capital projects 65333',
 1117.875: 'revenues over expenditures 134160',
 1221.96875: 'fund balance beginning of year 440069',
 1321.1875: 'fund balance end of year 574229',
 3018.34765625: 'the accompanying notes are an integral part of the financial statements'}

In [216]:
# Go through each line, check to see if it matches one of the form fields, then grab the numbers at the end of it
for line in document_line_dict.values():
	for label in table_form.keys():
		if re.match(label, line):
			if table_form[label] != -1:
				print('Duplicate field')
				break
			
			if line[-1] in ['_', '-']:
				table_form[label] = 0
			else:
				ending_numbers = re.search(r'\d+$', line)
				if ending_numbers:
					table_form[label] = int(ending_numbers.group())
				else:
					print('No numbers in line')
			break

In [217]:
table_form

{'property tax': 173246,
 'interest': 26247,
 'total revenues': 199493,
 'capital projects': 65333,
 'revenues over expenditures': 134160,
 'fund balance beginning of year': 440069,
 'fund balance end of year': 574229}

In [ ]:
# Do check sums
revenue_sum_match = table_form['property tax'] + table_form['interest'] == table_form['total revenues']
expenditure_match = table_form['total revenues'] - 